# CS498PS - Lab 5: Microphone Arrays

In this lab we will perform some simple microphone array processing. We will use the sound below:

[https://drive.google.com/uc?export=download&id=1emuGR4tlmemJ8RXSD1rWQwNx13h5X9VM ]

This is a recording from an 8-channel array. The microphones were placed at a distance of 0.1 meters from each other, and two simultaneous sounding sources were recorded. In the rest of this lab you will have to find where the sources are, and beamform so that you focus on each one separately.

## Part 1: Getting the Steering Vectors

In order to do any further processing on this array we will need to obtain a set of steering vectors. As you might recall the steering vectors encode the phase shift that each frequency undergoes between all the microphones of the array for a given source location.  Since we will be using a far-field model you will need to generate a steering vector for each frequency and source angle you want to check. We will assume that we will use 1024pt DFTs so that you need to generate steering vectors for 513 frequencies. You will also need to scan all angles from 0 to $\pi$. Since we won’t be making a continuous scan you can instead use 50 uniformly-spaced angles in that range.

Recall that the steering vector formula is:
$$v(\theta,m,k)=e^{-j\frac{(m-1)\cdot r \cdot cos(\theta)}{C} \frac{2\pi \cdot k}{N}}$$

where $N$ is the size of the DFT and $k$ is a frequency index, $R$ is the sample rate, $C$ the speed of sound (use 345 m/s), $r$ is the distance between the mics, $m$ is the mic index (1 to 8 in this case) and $\theta$ is the angle to check. Start by computing this with a simple triple loop, and later on revisit it and see if you can think of a simpler way to generate $v$.

In [7]:
import sys
print(sys.executable)
# !{sys.executable} -m pip install scipy
# from platform import python_version
# print(sys.executable)
# print(python_version())

/usr/bin/python3


In [1]:
# YOUR CODE HERE
import numpy as np
import math
import IPython
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
speedOfSound = 345
radianMin = 0 #0
radianMax = 180 #pi
r = 0.1
micMin = 1
micMax = 9
Nfreq = 513
DFTSize = 1024
rate, file = wavfile.read('array.wav')
# IPython.display.display( IPython.display.Audio( file, rate=rate))

#load file and get sampling rate
v = np.zeros((radianMax, micMax, Nfreq), dtype=complex)
for theta in range(radianMin,radianMax):
    for m in range(micMin, micMax):
        for k in range(0,Nfreq):
            v[theta,m,k] = np.exp(-1j*(((m-1)*r*math.cos(theta))/speedOfSound)*((2*np.pi*k)/DFTSize)*rate)
print(v[90])
print(v[0])
# raise NotImplementedError()

[[ 0.        +0.j          0.        +0.j          0.        +0.j
  ...  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 1.        +0.j          1.        +0.j          1.        +0.j
  ...  1.        +0.j          1.        +0.j
   1.        +0.j        ]
 [ 1.        +0.j          0.99984562+0.017571j    0.99938252+0.03513657j
  ... -0.8946685 +0.44673065j -0.90237988+0.43094146j
  -0.90981264+0.41501922j]
 ...
 [ 1.        +0.j          0.99614284+0.08774652j  0.9846011 +0.17481613j
  ...  0.67778238+0.73526257j  0.61065133+0.79189958j
   0.53880952+0.84242763j]
 [ 1.        +0.j          0.99444725+0.1052362j   0.97785069+0.2093037j
  ... -0.93485487-0.35503009j -0.89230184-0.45143928j
  -0.83983937-0.542835j  ]
 [ 1.        +0.j          0.99244463+0.12269338j  0.96989267+0.24353277j
  ...  0.99498803-0.09999408j  0.99973914+0.02283986j
   0.98938343+0.14532867j]]
[[ 0.        +0.j          0.        +0.j          0.        +0.j
  ...  0.        +0.j          

## Part 2: Localization

Now that we have the steering vectors we can perform some localization. In order to find where the sources are we will make a beamformer that “focuses” at each angle that we want to check and then returns the amount of acoustic energy that emanates from that point. Wherever there is a source we will see an energy bump. In order to perform the beamforming we will need to undo the phase shifts that are imposed on a sound from each angle. If we do so, for a signal that emanates from that angle we will appropriately phase shift the inputs so that they are perfectly synchronized over all channels. If we have the desired source perfectly synchronized over all channels and we add them together we will boost that source by a factor of 8 (the number of microphones). If this synchrony is not present we will get a lesser boost.

In order to undo the phase shift we simply need to apply the inverse steering vectors on the input. Perform an STFT of each channel with 1024 frequencies, a hop size of 256 and a Hann window (I hope you have already finished our code from Lab 1!). In order to apply the necessary phase shift on each channel you need to multiply each STFT spectrum (i.e. each column of the STFT output) with the conjugate of the steering vector that corresponds to its microphone $m$ and the angle $\theta$ that you want to measure. For each angle you want to measure, do this over all the microphones and sum the resulting spectrograms from all the channels. Once you do that measure the mean squared value of this sum and this will be your response from angle $\theta$. Do this over all angles and plot the overall response. The peaks of the resulting plot will reveal to you where the sources are.


In [2]:
# YOUR CODE HERE
rate, file = wavfile.read('crumble.wav')
def stft(input_sound, dft_size, hop_size, zero_pad, window):
    # YOUR CODE HERE
    padNum = math.ceil((len(input_sound)/dft_size)*dft_size-len(input_sound))
    pad = list(input_sound) + list (padNum*[0])
    pad = [0]*dft_size + pad + [0]*dft_size
    tempMatrix = []
    for i in range(0, len(pad)-dft_size+1, hop_size):
        tempMatrix.append(pad[i:i+dft_size]*window)
    return np.fft.rfft(tempMatrix, dft_size+zero_pad, axis=1) 

def istft( stft_output, dft_size, hop_size, zero_pad, window):
    # YOUR CODE HERE
    segments = np.fft.irfft(stft_output, axis=1)
    input_sound = []
    for i in range (0, len(stft_output), dft_size//hop_size):
        input_sound += list(segments[i]/window)
    input_sound = input_sound[:-zero_pad]
    return input_sound

# raise NotImplementedError()

In [3]:
sample_rate, input_sound = wavfile.read('./crumble.wav')
# sample_rate, input_sound = scipy.io.wavfile.read('./piano.wav')
# sample_rate, input_sound = scipy.io.wavfile.read('./speech.wav')
input_sound = input_sound.astype('float64')
input_sound /= 2**16 - 1

stftResult = stft(input_sound, dft_size, dft_size // 4, zero_pad, window)
inverseResult = istft(stftResult, dft_size, dft_size // 4, zero_pad, window)

sound(input_sound,rate=sample_rate, label='Original')
sound(inverseResult,rate=sample_rate, label='Inverse')

NameError: name 'dft_size' is not defined

## Part 3: Beamforming

Identify the angle of the two sources by looking at the peaks from the above result. Let’s call these $\theta_1$ and $\theta_2$. Now that you know where you want to focus the array, you can design two beamformers to focus on the two sources. The steering vectors that you need to use will be $v(\theta_1,:,:)$ and $v(\theta_2,:,:)$. Just as before you need to take each channel’s STFT, multiply each column with the conjugate of the steering vector that corresponds to all the channels and the selected angle to focus on, and then you simply add them all up. The resulting sum will the STFT of the focused output. Use your inverse STFT function to take this back to the time domain and verify that it indeed sounds better than any of the input channels.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()